In [1]:
import pandas as pd

In [3]:
path = r"C:\Users\gmart\Desktop\Spring 2024\PONI\jallison.csv"
df = pd.read_csv(path, index_col="Key")

properties_measured = [
    "TEM",
    "Atom probe tomography",
    "Optical micrograph",
    "SEM",
    "Macro image",
    "EBSD",
    "XRD",
    "EDS",
    "NAMLT",
    "Tensile behavior",
    "Shear strength behavior",
    "3PB",
    "Electrical conductivity",
    "PDP (potentiodynamic polarization)",
    "EIS (electrochemical impedance spectroscopic)",
    "Computational thermodynamics",
    "Line-intercept method",
    "fatigue behavior",
    "fatigue model"
]

focus_of_investigation = [
    "heat treatment",
    "temperature behavior",
    "preheated substrate",
    "dissimilar joining",
    "simulation",
    "corrosion",
    "neutron imaging",
    "layer thickness",
    "overlap",
    "fatigue"
]

In [4]:
df.dropna(subset=['Automatic Tags', 'Manual Tags'], inplace=True)

In [17]:
df["properties_measured"] = ""
df["focus_of_investigation"] = ""

In [18]:
df['article_id'] = df['Author'].str.split(';').str[0].str.strip()
df['article_id'] = df['article_id'] + df['Publication Year'].astype(str)

In [19]:
df["Automatic Tags"] = df['Automatic Tags'].astype('string')
df["Manual Tags"] = df['Manual Tags'].astype('string')
df["Tags"] = df["Automatic Tags"] +';'+ df["Manual Tags"]

In [20]:
temp_focus = []
temp_properties = []

def process_tags(tags):
    properties_found = []
    focus_found = []
    
    observation = tags.split(";")  
    
    for list_element in observation:
        if any(i in list_element for i in properties_measured):
            properties_found.append(list_element)
            
        if any(i in list_element for i in focus_of_investigation):
            focus_found.append(list_element)
    
    return ";".join(properties_found), ";".join(focus_found)

df[['Properties_Measured', 'Focus_of_Investigation']] = df['Tags'].apply(process_tags).apply(pd.Series)

In [21]:
df.drop(df.columns.difference(['article_id', 'Title', 'Properties_Measured', 'Focus_of_Investigation']), 1, inplace=True)

C:\Users\gmart\AppData\Local\Temp\ipykernel_44864\2828220499.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['article_id', 'Title', 'Properties_Measured', 'Focus_of_Investigation']), 1, inplace=True)


In [22]:
df.head()

,Title,article_id,Properties_Measured,Focus_of_Investigation
Key,,,,
I4YACKDU,The corrosion behavior and mechanical properti...,"Li, Yidi2023.0",EBSD; SEM; Tensile behavior; TEM; EDS; XRD; N...,heat treatment
HR3BEULB,Precipitation in AA6063 produced from swarf us...,"Babaniaris, Steven2022.0",SEM; TEM; EDS; Electrical conductivity,heat treatment
4XUI6X9F,The effect of anodization on the mechanical pr...,"Zhu, Ning2021.0",SEM; Tensile behavior; Optical micrograph; EDS,
N5B5H3UX,Microstructure-deformation relationship of add...,"Phillips, B. J.2019.0",Atom probe tomography; EBSD; TEM,
XVLHNYRZ,Process-structure-property relations for as-de...,"Mason, C. J.T.2021.0",EBSD; SEM; Tensile behavior; TEM; EDS,


# Testing Pyzotero

In [5]:
research_groups = {
    "Alabama": "ENTFKF9A",
    "BYU": "FHUA2AG6",
    "CSU": "J9P6QVL5",
    "Deakin": "FJDVBN8V",
    "ERDC": "A2IE9CCN",
    "JALLISON": "8R7A5FQF",
    "Jiangsu": "PIYWXUPX",
    "LSU": "BZPPUWWE",
    "Macao": "Q7J9TBZS",
    "Milano": "53EIUIDU",
    "Nanjing": "BXJRXGGL",
    "Northwester": "PZFMYYIG",
    "RIT": "IT7R9HI9",
    "Southwest": "8GR28UYH",
    "Tianjin": "YZFTU45C",
    "UNT": "K35CYECW",
    "UTK": "EKEA9F9A",
    "UTwente": "RIABIUZK",
    "UW Madison": "YWZYT6L9",
    "Yu Group": "A869GQQS"
}

In [7]:
from pyzotero import zotero

library_id = 5025296
api_key = "98LqcOAUtBstPC5XJe4699fz"
library_type = 'group'

zot = zotero.Zotero(library_id, library_type, api_key)

In [13]:
def extract_item_info(collection_id):
    items = zot.collection_items(collection_id)
    item_info = {}
    for item in items:
        item_id = item['key']
        authors = item['data'].get('creators', [])
        author_names = ', '.join([author['lastName'] for author in authors])
        year = item['data'].get('date', '')
        title = item['data'].get('title', '')
        tags = [tag['tag'] for tag in item['data'].get('tags', [])]
        item_info[item_id] = {
            'Author': author_names,
            'Year': year,
            'Title': title,
            'Tags': ', '.join(tags)
        }
    return item_info

In [14]:
data = []

for group_name, group_id in research_groups.items():
    print("Research Group:", group_name)
    collection_info = extract_item_info(group_id)
    for item_id, info in collection_info.items():
        data.append({
            'Research Group': group_name,
            'Item ID': item_id,
            'Author': info['Author'],
            'Year': info['Year'],
            'Title': info['Title'],
            'Tags': info['Tags']
        })
        
df = pd.DataFrame(data)

Research Group: Alabama
Research Group: BYU
Research Group: CSU
Research Group: Deakin
Research Group: ERDC
Research Group: JALLISON
Research Group: Jiangsu
Research Group: LSU
Research Group: Macao
Research Group: Milano
Research Group: Nanjing
Research Group: Northwester
Research Group: RIT
Research Group: Southwest
Research Group: Tianjin
Research Group: UNT
Research Group: UTK
Research Group: UTwente
Research Group: UW Madison
Research Group: Yu Group


In [17]:
df.head(10)

,Research Group,Item ID,Author,Year,Title,Tags
0,Alabama,GR6VTSQG,,,Merritt et al. - 2023 - Nonlinear Temperature ...,
1,Alabama,7LEF9WPB,"Merritt, Cousin, Yoon",2023-11-08,Nonlinear Temperature Control of Additive Fric...,
2,BYU,9DEMMH44,"Glenn, Dean, Wright, Hovanski, Hovanski, Sato,...",2023,Closed-Loop PID Temperature Control of Additiv...,"AFSD, Additive friction stir deposition, PID c..."
3,BYU,5UT9Z263,,,,
4,CSU,R7LBHDZ8,"Li, Zhang, Wang, Lai, Yang, Li",2023-11-06,Microstructure and mechanical properties of Al...,"AA2195, Additive friction stir deposition, Al-..."
5,CSU,W7AGEBLC,"Zhang, Ye, Li, Wang, Lai, Li",2023/1,Effect of Heat Treatment States of Feedstock o...,"AA2219, EBSD, FRC002, Fractography, SEM, Tensi..."
6,CSU,FTWMBPBN,"Gong, Li, Zhang, Wang, Liu, Zeng, Ma, Yang, La...",2023-09-01,Influence of Preheating Temperature on the Mic...,"6061/TA1 composite, AA6061, Cladding, EDS, FRC..."
7,CSU,I4YACKDU,"Li, Yang, Zhang, Wang, Gong, Lai, Li, Teng",2023-04-01,The corrosion behavior and mechanical properti...,"AA5083, Corrosion, EBSD, EDS, FRC002, Fractogr..."
8,CSU,DAQ8YIJT,,,Full Text PDF,
9,CSU,VACUGVZC,"Wang, Li, Zhang, Gong, Lai, Li",2023,Preheating-assisted solid-state friction stir ...,"to-digitize, ⚠️ Invalid DOI"


# Combining Pyzotero and Application